In [1]:
#Dependencies
import pandas as pd
import numpy as np
from sportsreference.nba.schedule import Schedule
from sportsreference.nba.teams import Teams 
from datetime import date, datetime

In [2]:
#defining advanced rest

def adv_rest(team,year):
    #creating the rest day calc function

    def rest_days(team,year):

        #creating the team schedule
        schedule = Schedule(team, year)

        dates = []
        results = []
        locations = []
        game_nums = []
        opponents = []



        for game in schedule:

            if game.playoffs == False :
                dates.append(game.datetime)
                results.append(game.result)
                locations.append(game.location)
                game_nums.append(game.game)
                opponents.append(game.opponent_abbr)

        #creating team dictionary
        team_dict = {
            "dates": dates,
            "results": results,
            "locations": locations,
            "game_nums": game_nums,
            "opponent": opponents
        }

        #creating bucks data frame
        team_df = pd.DataFrame(team_dict)
        team_df

        #finding the number of games between each game
        team_df["date_diff"] = 0

        for i in range(1,len(team_df)):    
            team_df["date_diff"][i] = team_df["dates"][i] - team_df["dates"][(i-1)]

        #inserting value for the date diff in the first game
        #using 10 as a standard for the date difference of the first game (most impactful players don't play preseason anyway)
        team_df["date_diff"][0] = 10

        #converting to a time delta
        team_df["date_diff"][0] = pd.to_timedelta(team_df["date_diff"][0], unit = "D")

        #Converting the date difference series back into an integer for easier comparisons
        for day in range(0,len(team_df["date_diff"])):
            team_df["date_diff"][day] = team_df["date_diff"][day] / np.timedelta64(1, 'D')

        #date diff is only the difference between the two games
        #creating a column to represent the "rest" days between the games
        #"rest" in this case is the days between each game where a game is not being played and does not account for practices
        team_df["rest"] = team_df["date_diff"] - 1 

        return team_df
    
    #passing team to the inner function
    team_df = rest_days(team,year)
    
    opp_diff = []
    opp_rest = []
    
    #iterate through the games
    for i in range(0,len(team_df)):
        
        #for each opponent create a rest dataframe
        opp_df = rest_days(str(team_df['opponent'][i]),year)
        
        #append the rest to the lists
        opp_diff.append(opp_df['date_diff'][i])
        opp_rest.append(opp_df['rest'][i])
    
    #new columns for opponents' values
    team_df['opp_date_diff'] = opp_diff
    team_df['opp_rest'] = opp_rest
    
    #creating a rest difference column
    team_df['rest_diff'] = team_df['rest'] - team_df['opp_rest']
    
    return team_df
    

In [3]:
#test1

wolves_df = adv_rest('MIN',2019)

print(len(wolves_df))
wolves_df.head(20)

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

82


,dates,results,locations,game_nums,opponent,date_diff,rest,opp_date_diff,opp_rest,rest_diff
0,2018-10-17,Loss,Away,1,SAS,10,9,10.0,9.0,0
1,2018-10-19,Win,Home,2,CLE,2,1,2.0,1.0,0
2,2018-10-20,Loss,Away,3,DAL,1,0,2.0,1.0,-1
3,2018-10-22,Win,Home,4,IND,2,1,2.0,1.0,0
4,2018-10-24,Loss,Away,5,TOR,2,1,2.0,1.0,0
5,2018-10-26,Loss,Home,6,MIL,2,1,1.0,0.0,1
6,2018-10-29,Win,Home,7,LAL,3,2,2.0,1.0,1
7,2018-10-31,Win,Home,8,UTA,2,1,2.0,1.0,0
8,2018-11-02,Loss,Away,9,GSW,2,1,2.0,1.0,0
9,2018-11-04,Loss,Away,10,POR,2,1,1.0,0.0,1


In [4]:
#creating a league dataset 2018-2019 regular season

#getting a list of team abbreviations to pass to the function

team_abbrv = []

teams = Teams()

for team in teams:
    team_abbrv.append(team.abbreviation)
    print(team.abbreviation)

MIL
BRK
WAS
UTA
POR
DEN
NOP
LAC
SAC
GSW
IND
ATL
MEM
PHI
PHO
DAL
TOR
BOS
MIN
SAS
CHI
HOU
CHO
LAL
DET
MIA
NYK
OKC
ORL
CLE


In [7]:
#passing the team abbreviations to the function

for team in team_abbrv:
    
    
    print('-----'*10)
    
    team_df = adv_rest(str(team),2019)
    team_df['abbrv'] = str(team)
    print(team_df)

--------------------------------------------------


C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

        dates results locations  game_nums opponent date_diff rest  \
0  2018-10-17     Win      Away          1      CHO        10    9   
1  2018-10-19     Win      Home          2      IND         2    1   
2  2018-10-22     Win      Home          3      NYK         3    2   
3  2018-10-24     Win      Home          4      PHI         2    1   
4  2018-10-26     Win      Away          5      MIN         2    1   
..        ...     ...       ...        ...      ...       ...  ...   
77 2019-04-01     Win      Away         78      BRK         1    0   
78 2019-04-04     Win      Away         79      PHI         3    2   
79 2019-04-06    Loss      Home         80      BRK         2    1   
80 2019-04-07     Win      Home         81      ATL         1    0   
81 2019-04-10    Loss      Home         82      OKC         3    2   

    opp_date_diff  opp_rest rest_diff abbrv  
0            10.0       9.0         0   MIL  
1             2.0       1.0         0   MIL  
2             1.0    

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

        dates results locations  game_nums opponent date_diff rest  \
0  2018-10-17    Loss      Away          1      DET        10    9   
1  2018-10-19     Win      Home          2      NYK         2    1   
2  2018-10-20    Loss      Away          3      IND         1    0   
3  2018-10-24     Win      Away          4      CLE         4    3   
4  2018-10-26    Loss      Away          5      NOP         2    1   
..        ...     ...       ...        ...      ...       ...  ...   
77 2019-04-01    Loss      Home         78      MIL         2    1   
78 2019-04-03    Loss      Home         79      TOR         2    1   
79 2019-04-06     Win      Away         80      MIL         3    2   
80 2019-04-07     Win      Away         81      IND         1    0   
81 2019-04-10     Win      Home         82      MIA         3    2   

    opp_date_diff  opp_rest rest_diff abbrv  
0            10.0       9.0         0   BRK  
1             2.0       1.0         0   BRK  
2             1.0    

...

        dates results locations  game_nums opponent date_diff rest  \
0  2018-10-16    Loss      Away          1      GSW        10    9   
1  2018-10-19    Loss      Away          2      LAC         3    2   
2  2018-10-21    Loss      Home          3      SAC         2    1   
3  2018-10-25    Loss      Home          4      BOS         4    3   
4  2018-10-28     Win      Home          5      PHO         3    2   
..        ...     ...       ...        ...      ...       ...  ...   
77 2019-04-02     Win      Home         78      LAL         2    1   
78 2019-04-05     Win      Home         79      DET         3    2   
79 2019-04-07     Win      Away         80      MIN         2    1   
80 2019-04-09     Win      Home         81      HOU         2    1   
81 2019-04-10     Win      Away         82      MIL         1    0   

    opp_date_diff  opp_rest rest_diff abbrv  
0            10.0       9.0         0   OKC  
1             2.0       1.0         1   OKC  
2             2.0    

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

        dates results locations  game_nums opponent date_diff rest  \
0  2018-10-17     Win      Home          1      MIA        10    9   
1  2018-10-19    Loss      Home          2      CHO         2    1   
2  2018-10-20    Loss      Away          3      PHI         1    0   
3  2018-10-22     Win      Away          4      BOS         2    1   
4  2018-10-25    Loss      Home          5      POR         3    2   
..        ...     ...       ...        ...      ...       ...  ...   
77 2019-04-01    Loss      Away         78      TOR         2    1   
78 2019-04-03     Win      Home         79      NYK         2    1   
79 2019-04-05     Win      Home         80      ATL         2    1   
80 2019-04-07     Win      Away         81      BOS         2    1   
81 2019-04-10     Win      Away         82      CHO         3    2   

    opp_date_diff  opp_rest rest_diff abbrv  
0            10.0       9.0         0   ORL  
1             2.0       1.0         0   ORL  
2             2.0    

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

        dates results locations  game_nums opponent date_diff rest  \
0  2018-10-17    Loss      Away          1      TOR        10    9   
1  2018-10-19    Loss      Away          2      MIN         2    1   
2  2018-10-21    Loss      Home          3      ATL         2    1   
3  2018-10-24    Loss      Home          4      BRK         3    2   
4  2018-10-25    Loss      Away          5      DET         1    0   
..        ...     ...       ...        ...      ...       ...  ...   
77 2019-04-01    Loss      Away         78      PHO         2    1   
78 2019-04-04    Loss      Away         79      SAC         3    2   
79 2019-04-05    Loss      Away         80      GSW         1    0   
80 2019-04-07    Loss      Home         81      SAS         2    1   
81 2019-04-09    Loss      Home         82      CHO         2    1   

    opp_date_diff  opp_rest rest_diff abbrv  
0            10.0       9.0         0   CLE  
1             2.0       1.0         0   CLE  
2             2.0    

In [23]:
rest_19 = adv_rest('MIL',2019)
rest_19['abbrv'] = str('MIL')

count = 1


for team in team_abbrv[1:]:
  
    print(count)
    
    team_df = adv_rest(str(team),2019)
    team_df['abbrv'] = str(team)
    rest_19 = pd.concat([rest_19,team_df], ignore_index=True)
    
    count +=1

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:56:

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [24]:
rest_19

,dates,results,locations,game_nums,opponent,date_diff,rest,opp_date_diff,opp_rest,rest_diff,abbrv
0,2018-10-17,Win,Away,1,CHO,10,9,10.0,9.0,0,MIL
1,2018-10-19,Win,Home,2,IND,2,1,2.0,1.0,0,MIL
2,2018-10-22,Win,Home,3,NYK,3,2,1.0,0.0,2,MIL
3,2018-10-24,Win,Home,4,PHI,2,1,3.0,2.0,-1,MIL
4,2018-10-26,Win,Away,5,MIN,2,1,2.0,1.0,0,MIL
...,...,...,...,...,...,...,...,...,...,...,...
2455,2019-04-01,Loss,Away,78,PHO,2,1,2.0,1.0,0,CLE
2456,2019-04-04,Loss,Away,79,SAC,3,2,2.0,1.0,1,CLE
2457,2019-04-05,Loss,Away,80,GSW,1,0,2.0,1.0,-1,CLE
2458,2019-04-07,Loss,Home,81,SAS,2,1,2.0,1.0,0,CLE


In [26]:
test_data = rest_19.copy()

In [38]:
#converting types
test_data['date_diff'] = test_data['date_diff'].astype(str).astype(int)

test_data['rest'] = test_data['rest'].astype(int)

In [46]:
#description
test_data.describe()

,game_nums,date_diff,rest,opp_date_diff,opp_rest
count,2460.000000,2460.000000,2460.000000,2460.000000,2460.000000
mean,41.500000,2.252846,1.252846,2.210569,1.210569
std,23.674413,1.304531,1.304531,1.319748,1.319748
min,1.000000,1.000000,0.000000,1.000000,0.000000
25%,21.000000,2.000000,1.000000,2.000000,1.000000
50%,41.500000,2.000000,1.000000,2.000000,1.000000
75%,62.000000,2.000000,1.000000,2.000000,1.000000
max,82.000000,10.000000,9.000000,10.000000,9.000000


In [ ]:
#exporting as csv
